In [ ]:
!pip install git+https://github.com/toncenter/tvm_valuetypes.git

In [1]:
#%load_ext autoreload
#%autoreload 2
from tvm_python import PyTVM, method_name_to_id, code_disasseble
import codecs
from tvm_valuetypes import Cell, deserialize_boc
from dateutil.parser import parse 

In [2]:
def fix_op(item):
    return item.split('\t')[1].replace('execute ', '').replace('\n', '')

# Just code

In [3]:
PyTVM(code="te6ccgEBAQEAAwAAAnE=").run_vm()

[1]

### Stack is not reversed ;)

In [4]:
SUB = "te6ccgEBAQEAAwAAAqE="
a = 3
b = 1

tvm = PyTVM(code=SUB)
tvm.set_stack([a, b])
tvm.run_vm()

[2]

#### Balance can be really big integer

In [7]:
tvm = PyTVM(code=SUB)

tvm.set_c7(address="EQCI5yVsMccJDqrKH9knYAp0avK5fh-UBZ5Ns1xWOvIA2UKl", # some contracts may use MYADDR in code
           balanceGrams="9951186000", # or trying get own balance from c7
           randseed="1") # or maybe you use random in your contract :)

# Disassembler

In [8]:
code_disasseble("te6ccgEBAQEAAwAAAnE=")

'PUSHINT 1\n'

# Easy test examples

In [9]:
# Simple PUSHINT 1
# You can define log_level if you want more stuff
tvm_result = PyTVM(code="te6ccgEBAQEAAwAAAnE=", log_level=4).run_vm()
assert tvm_result[0] == 1

print(f"Got 1 from TVM: {tvm_result[0]}")

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 8810E73F1AA91EC4618658513623A56A18B5E2B2FDCA2F2A22000D2A14001B89
DEBUG: Use code: 8810E73F1AA91EC4618658513623A56A18B5E2B2FDCA2F2A22000D2A14001B89
DEBUG: Load cp0
[ 4][t 0][2022-10-29 22:46:16.953755][arithops.cpp:34]	execute PUSHINT 1

[ 4][t 0][2022-10-29 22:46:16.954070][vm.cpp:478]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #0
Got 1 from TVM: 1


 [ ] 
 [ 1 ] 


In [10]:
tvm_result

[1]

In [11]:
# c3 is continuation, but we just serialize it to base64 BOC
# PUSHINT 1 PUSH c3
tvm_result = PyTVM(code="te6ccgEBAQEABQAABnHtQw==").run_vm()
tvm_result

[1,
 {'type': 'continuation',
  'value': 'te6ccuEBAgEADgASHAEMBAAAAAYAAQAGce1DB7kzoA=='}]

In [12]:
cell = Cell()
b64 = tvm_result[1]['value']
boc = codecs.decode(codecs.encode(b64, 'utf8'), 'base64')
deserialize_boc(boc)

<Cell refs_num: 1, data: b'\x04\x00\x00\x00\x06\x00'>

In [13]:
# c7 push (tuple)
tvm_result = PyTVM(code="te6ccgEBAQEABAAABO1H").run_vm()
tvm_result[0] # just default c7

[[124711402,
  0,
  0,
  0,
  0,
  0,
  0,
  [101000000000, None],
  {'type': 'cellSlice', 'value': 'te6ccuEBAQEACgAUABAAAAAAAAAAALfVArM='},
  None]]

## Change c7

In [14]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H")

In [17]:
tvm.set_c7(translt="100",
           unixtime=179, 
           globalConfig="te6ccgEBAQEABAAABO1H",
           randseed="100", 
           address="EQBq5JLG8-1juc-N95IFBnOM2NGrcJsMoFYFoWH-DaVJ2lVw",
           balanceGrams=str(100000000))

In [18]:
tvm.run_vm()

[[[124711402,
   0,
   0,
   179,
   0,
   100,
   100,
   [100000000, None],
   {'type': 'cellSlice',
    'value': 'te6ccuEBAQEAJABIAEOADVySWN59rHc58b7yQKDOcZsaNW4TYZQKwLQsP8G0qTtQ5qaveg=='},
   {'type': 'cell', 'value': 'te6ccuEBAQEABAAIAATtR+R0b2c='}]]]

## Get detailed result

In [19]:
print("Exit code: ", tvm.exit_code,
"\nC5: ", tvm.actions, 
"\nGas credit: ",tvm.gas_credit, 
"\nGas used: ",tvm.gas_used, 
"\nC4: ",tvm.new_data, 
"\nVM Steps: ",tvm.vm_steps,
"\nVM final hash: ",tvm.vm_final_state_hash,
"\nVM init hash: ",tvm.vm_init_state_hash,
"\nVM success: ",tvm.success)

Exit code:  -1 
C5:  te6ccuEBAQEAAgAEAABmLc6k 
Gas credit:  0 
Gas used:  31 
C4:  te6ccuEBAQEAAgAEAABmLc6k 
VM Steps:  2 
VM final hash:  0000000000000000000000000000000000000000000000000000000000000000 
VM init hash:  0000000000000000000000000000000000000000000000000000000000000000 
VM success:  True


## Set stack

In [20]:
# c7 push (tuple)
tvm = PyTVM(code="te6ccgEBAQEABAAABO1H", log_level=6)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C


In [21]:
# actually python3 & TON have really big integers :))
tvm.set_stack([
    190009329293929392939239293293929329392949949949299008888888880999999888, 
    {'type': "cell",
     'value': 'te6ccgEBAQEABAAABO1H'}, 
    [{'type': "cellSlice",
     'value': 'te6ccgEBAQEABAAABO1H'},
     12312312313213132,
     [1231232132, 1321231231, 57, 179]
    ]])

In [22]:
tvm.run_vm()

DEBUG: Use code: 2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C
DEBUG: Load cp0
[ 4][t 0][2022-10-29 22:46:38.643558][contops.cpp:694]	execute PUSH c7

[ 4][t 0][2022-10-29 22:46:38.643977][vm.cpp:478]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #3
DEBUG: Parse stack item #2
DEBUG: Parse stack item #1
DEBUG: Parse stack item #0


 [ 190009329293929392939239293293929329392949949949299008888888880999999888 C{2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C} [CS{Cell{0004ed47} bits: 0..16; refs: 0..0} 12312312313213132 [1231232132 1321231231 57 179]] ] 
 [ 190009329293929392939239293293929329392949949949299008888888880999999888 C{2F306EC2BCE112068F472B947E0F6524ADA3F47624DE1EBB0C2BC9304661640C} [CS{Cell{0004ed47} bits: 0..16; refs: 0..0} 12312312313213132 [1231232132 1321231231 57 179]] [[124711402 0 0 0 0 0 0 (101000000000) CS{Cell{00100000000000000000} bits: 0..64; refs: 0..0} ()]] ] 


[190009329293929392939239293293929329392949949949299008888888880999999888,
 {'type': 'cell', 'value': 'te6ccuEBAQEABAAIAATtR+R0b2c='},
 [{'type': 'cellSlice', 'value': 'te6ccuEBAQEABAAIAATtR+R0b2c='},
  12312312313213132,
  [1231232132, 1321231231, 57, 179]],
 [[124711402,
   0,
   0,
   0,
   0,
   0,
   0,
   [101000000000, None],
   {'type': 'cellSlice', 'value': 'te6ccuEBAQEACgAUABAAAAAAAAAAALfVArM='},
   None]]]

## C4 (Data) usage example

In [23]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=2)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449


In [24]:
# You can see, that 179 was loaded from c4, and cellSlice on stack
tvm.run_vm()

DEBUG: Use code: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Load cp0
[ 4][t 0][2022-10-29 22:46:40.080243][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-29 22:46:40.080555][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-29 22:46:40.080704][cellops.cpp:940]	execute LDU 64

[ 4][t 0][2022-10-29 22:46:40.080803][vm.cpp:478]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #1
DEBUG: Parse stack item #0


[179, {'type': 'cellSlice', 'value': 'te6ccuEBAQEAAgAEAABmLc6k'}]

## Gas Limit 

In [25]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=2)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449


In [26]:
tvm.set_gasLimit(26) # You can also pass max gas set_gasLimit(gas_limit=26, gas_max=26)
tvm.run_vm() # out of gas

DEBUG: Use code: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Load cp0
[ 4][t 0][2022-10-29 22:46:41.861183][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-29 22:46:41.861341][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-29 22:46:41.861745][vm.cpp:516]	unhandled out-of-gas exception: gas consumed=144, limit=26

DEBUG: VM terminated with exit code 13
DEBUG: Parse stack item #0


[144]

## Libs 

This is just load example, they will pass to VM, but I'm little bussy to get code of loading libs, check out [akifoq](https://github.com/akifoq) github

In [27]:
tvm.set_libs([{'type': 'cell', 'value': 'te6ccgEBAQEABwAACu1E0NM/'}])

## Stack dump log Level

In [28]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=4)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449


In [29]:
tvm.run_vm()

DEBUG: Use code: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Load cp0
[ 4][t 0][2022-10-29 22:46:44.277603][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-29 22:46:44.277740][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-29 22:46:44.277863][cellops.cpp:940]	execute LDU 64

[ 4][t 0][2022-10-29 22:46:44.277965][vm.cpp:478]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #1
DEBUG: Parse stack item #0


 [ ] 
 [ C{E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449} ] 
 [ CS{Cell{001000000000000000b3} bits: 0..64; refs: 0..0} ] 
 [ 179 CS{Cell{001000000000000000b3} bits: 64..64; refs: 0..0} ] 


[179, {'type': 'cellSlice', 'value': 'te6ccuEBAQEAAgAEAABmLc6k'}]

## Allow debug & same c3

In [30]:
# C4 PUSH CTOS 64 LDU
tvm = PyTVM(code="te6ccgEBAQEABwAACu1E0NM/", 
            data="te6ccgEBAQEACgAAEAAAAAAAAACz", 
            log_level=4,
            allow_debug=True,
            same_c3=False)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449


In [31]:
tvm.run_vm()

DEBUG: Use code: 893C09B4BEE71FD2D35901C5D2BE014A40E6D5613F3DAF187E1345201C5A5E11
DEBUG: Load cp0
[ 4][t 0][2022-10-29 22:46:46.168985][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-29 22:46:46.169131][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-29 22:46:46.169230][cellops.cpp:940]	execute LDU 64

[ 4][t 0][2022-10-29 22:46:46.169376][vm.cpp:478]	execute implicit RET

DEBUG: VM terminated with exit code -1
DEBUG: Parse stack item #1
DEBUG: Parse stack item #0


 [ ] 
 [ C{E2851C5358AF7F0B5C101ED925CAC97CE1F59B117C6C8E1B9791A0B618C7C449} ] 
 [ CS{Cell{001000000000000000b3} bits: 0..64; refs: 0..0} ] 
 [ 179 CS{Cell{001000000000000000b3} bits: 64..64; refs: 0..0} ] 


[179, {'type': 'cellSlice', 'value': 'te6ccuEBAQEAAgAEAABmLc6k'}]

## Accout get methods

In [32]:
# get account code & data from dton.io
tvm = PyTVM(code="te6ccuECDAEAAikAABoAJAAuADYAQACWAZYClAKuAv4DzARSART/APSkE/S88sgLAQIBIAIDAgFIBAUABPIwAgLNBgcAUaA4WdqJoaYBpj/0gfSB9IH0AahhofSB9AH0gfQAYQQgjJKwoBWAAqsBAvfQDoaYGAuNhJL4JwfSAYdqJoaYBpj/0gfSB9IH0AahgTYAD5aMoRa6ThAVnHIBkcHCmg44LJL4RwKKJjgvlw+gJpj8EIAonGyIldeXD66Z+Y/SAYICsDZGWACuWPqAHniwDniwDniwD9AWZk9qpwGxPjgHGBA+mP6Z+YEMCAkB92YIQO5rKAFJgoFIwvvLhwiTQ+kD6APpA+gAwU5KhIaFQh6EWoFKQcIAQyMsFUAPPFgH6AstqyXH7ACXCACXXScICsI4XUEVwgBDIywVQA88WAfoCy2rJcfsAECOSNDTiWnCAEMjLBVADzxYB+gLLaslx+wBwIIIQX8w9FILABY3EDhHZRRDMHDwBQFKwAGSXwvgIcACnzEQSRA4R2AQJRAkECPwBeA6wAPjAl8JhA/y8AoAyoIQO5rKABi+8uHJU0bHBVFSxwUVsfLhynAgghBfzD0UIYAQyMsFKM8WIfoCy2rLHxnLPyfPFifPFhjKACf6AhfKAMmAQPsAcQZQREUVBsjLABXLH1ADzxYBzxYBzxYB+gLMye1UAIIhgBjIywUqzxYh+gLLassfE8s/I88WUAPPFsoAIfoCygDJgwb7AHFVUAbIywAVyx9QA88WAc8WAc8WAfoCzMntVHEzQWc=", 
            data="te6ccuECAgEAAKEAAKIBQgGbMateFUAFhO5hst/wg3EW0Py1B42TlkvL6cBf1qFBsb/KXWpD4YgAdP2guq8i0ybWx7E2zTGM2WdwO9WauK3yzhASGFCjzN4wSMCWvrAEAQCbgBRyaww+87XrNCetowXCyAQhgF8xx74x+06XHrVig1fjCujOry8BAANKN8ZnM0OzYOENTkOEg7VVgFog5fBWdCtqQro1MRmU1dGdXl4CusI7qA==", 
            log_level=4)

DEBUG: Start parse code
DEBUG: Code parsed success
Code loaded: 8278F4C5233DE6FBEDC969AF519344A7A9BFFC544856DBA986A95C0BCF8571C9
DEBUG: Start parse data
DEBUG: Data parsed success
Data loaded: 43D29B5E2CC2AD75F4FFC97C064E1B1EE126F11195B5ECE506B28FFDEDDC6AE6


In [35]:
tvm.set_c7(address="EQCI5yVsMccJDqrKH9knYAp0avK5fh-UBZ5Ns1xWOvIA2UKl", # some contracts may use MYADDR in code
           balanceGrams=str(100000000), # or trying get own balance from c7
           randseed="1") # or maybe you use random in your contract :)

In [36]:
method_name_to_id("get_sale_data") # get methods have own numbers :)

72748

In [37]:
tvm.set_stack([method_name_to_id("get_sale_data")]) # long story short, you need to pass method_id to stack

In [38]:
tvm.run_vm() # this is seqno of this wallet :)

DEBUG: Use code: 8278F4C5233DE6FBEDC969AF519344A7A9BFFC544856DBA986A95C0BCF8571C9
DEBUG: Load cp0
[ 4][t 0][2022-10-29 22:46:55.029593][contops.cpp:1129]	execute SETCP 0

[ 4][t 0][2022-10-29 22:46:55.029687][dictops.cpp:631]	execute DICTPUSHCONST 19 (xC_,1)

[ 4][t 0][2022-10-29 22:46:55.029835][dictops.cpp:595]	execute DICTIGETJMPZ

[ 4][t 0][2022-10-29 22:46:55.030395][contops.cpp:694]	execute PUSH c4

[ 4][t 0][2022-10-29 22:46:55.030534][cellops.cpp:875]	execute CTOS

[ 4][t 0][2022-10-29 22:46:55.030792][cellops.cpp:940]	execute LDU 1

[ 4][t 0][2022-10-29 22:46:55.030977][cellops.cpp:940]	execute LDU 32

[ 4][t 0][2022-10-29 22:46:55.033808][tonops.cpp:532]	execute LDMSGADDR

[ 4][t 0][2022-10-29 22:46:55.034158][tonops.cpp:532]	execute LDMSGADDR

[ 4][t 0][2022-10-29 22:46:55.036017][tonops.cpp:532]	execute LDMSGADDR

[ 4][t 0][2022-10-29 22:46:55.036343][tonops.cpp:443]	execute LDGRAMS

[ 4][t 0][2022-10-29 22:46:55.036662][cellops.cpp:971]	execute LDREF

[ 4][t 0][2022-10-29 

 [ 72748 ] 
 [ 72748 ] 
 [ 72748 C{C631CA807ADDEBA16F1FCF99863574F748D8C5A4B74BD6E6BE8E26D289BE6CB7} 19 ] 
 [ ] 
 [ C{43D29B5E2CC2AD75F4FFC97C064E1B1EE126F11195B5ECE506B28FFDEDDC6AE6} ] 
 [ CS{Cell{019b31ab5e15400584ee61b2dff0837116d0fcb5078d93964bcbe9c05fd6a141b1bfca5d6a43e1880074fda0baaf22d326d6c7b136cd318cd967703bd59ab8adf2ce10121850a3ccde3048c096beb004} bits: 0..621; refs: 0..1} ] 
 [ 0 CS{Cell{019b31ab5e15400584ee61b2dff0837116d0fcb5078d93964bcbe9c05fd6a141b1bfca5d6a43e1880074fda0baaf22d326d6c7b136cd318cd967703bd59ab8adf2ce10121850a3ccde3048c096beb004} bits: 1..621; refs: 0..1} ] 
 [ 0 1666628650 CS{Cell{019b31ab5e15400584ee61b2dff0837116d0fcb5078d93964bcbe9c05fd6a141b1bfca5d6a43e1880074fda0baaf22d326d6c7b136cd318cd967703bd59ab8adf2ce10121850a3ccde3048c096beb004} bits: 33..621; refs: 0..1} ] 
 [ 0 1666628650 CS{Cell{019b31ab5e15400584ee61b2dff0837116d0fcb5078d93964bcbe9c05fd6a141b1bfca5d6a43e1880074fda0baaf22d326d6c7b136cd318cd967703bd59ab8adf2ce10121850a3ccde3048c096beb004} bits:

[1179211856,
 0,
 1666628650,
 {'type': 'cellSlice',
  'value': 'te6ccuEBAQEAJABIAEOACwncw2W/4QbiLaH5ag8bJyyXl9OAv61Cg2N/lLrUh8MQhD770g=='},
 {'type': 'cellSlice',
  'value': 'te6ccuEBAQEAJABIAEOAB0/aC6ryLTJtbHsTbNMYzZZ3A71Zq4rfLOEBIYUKPM3wm/BsZg=='},
 {'type': 'cellSlice', 'value': 'te6ccuEBAQEAAwAGAAEgx7f2dg=='},
 9999000000000,
 {'type': 'cellSlice',
  'value': 'te6ccuEBAQEAJABIAEOAFHJrDD7ztes0J62jBcLIBCGAXzHHvjH7TpcetWKDV+MQHJgoPA=='},
 499950000000,
 {'type': 'cellSlice',
  'value': 'te6ccuEBAQEAJABIAEOAAaUb4zOZodmwcIanIcJB2qrALRBy+Cs6FbUhXRqYjMpwrjERzg=='},
 499950000000]

a3ccde3048c096beb004} bits: 33..300; refs: 0..0} CS{Cell{019b31ab5e15400584ee61b2dff0837116d0fcb5078d93964bcbe9c05fd6a141b1bfca5d6a43e1880074fda0baaf22d326d6c7b136cd318cd967703bd59ab8adf2ce10121850a3ccde3048c096beb004} bits: 300..567; refs: 0..0} CS{Cell{019b31ab5e15400584ee61b2dff0837116d0fcb5078d93964bcbe9c05fd6a141b1bfca5d6a43e1880074fda0baaf22d326d6c7b136cd318cd967703bd59ab8adf2ce10121850a3ccde3048c096beb004} bits: 567..569; refs: 0..0} 9999000000000 CS{Cell{019b31ab5e15400584ee61b2dff0837116d0fcb5078d93964bcbe9c05fd6a141b1bfca5d6a43e1880074fda0baaf22d326d6c7b136cd318cd967703bd59ab8adf2ce10121850a3ccde3048c096beb004} bits: 621..621; refs: 0..1} ] 
 [ 0 1666628650 CS{Cell{019b31ab5e15400584ee61b2dff0837116d0fcb5078d93964bcbe9c05fd6a141b1bfca5d6a43e1880074fda0baaf22d326d6c7b136cd318cd967703bd59ab8adf2ce10121850a3ccde3048c096beb004} bits: 33..300; refs: 0..0} CS{Cell{019b31ab5e15400584ee61b2dff0837116d0fcb5078d93964bcbe9c05fd6a141b1bfca5d6a43e1880074fda0baaf22d326d6c7b136cd318cd967703

## Method names

In [39]:
smart_methods = {}

for name in ["dnsresolve", "get_jetton_data", "get_wallet_address", "get_collection_data",
             "get_nft_address_by_index", "get_nft_data", "get_royalty_params", "hello_world",
             "get_owner", "seqno", "get_public_key", "get_version",
             "get_marketplace_address", "get_sale_data", "get_plugin_list", "is_plugin_installed",
             "get_subwallet_id", "get_editor", "get_is_closed", 
             'get_reveal_mode', 'get_reveal_data', 
             
             'get_full_domain', 'get_username', 'get_auction_info', 
             'get_telemint_token_name', 'get_telemint_auction_state', 
             'get_telemint_auction_config', 'get_offer_data',
            ]:

    smart_methods[name] = method_name_to_id(name)

In [40]:
smart_methods

{'dnsresolve': 123660,
 'get_jetton_data': 106029,
 'get_wallet_address': 103289,
 'get_collection_data': 102491,
 'get_nft_address_by_index': 92067,
 'get_nft_data': 102351,
 'get_royalty_params': 128411,
 'hello_world': 113531,
 'get_owner': 80293,
 'seqno': 85143,
 'get_public_key': 78748,
 'get_version': 82320,
 'get_marketplace_address': 91689,
 'get_sale_data': 72748,
 'get_plugin_list': 107653,
 'is_plugin_installed': 76407,
 'get_subwallet_id': 81467,
 'get_editor': 90228,
 'get_is_closed': 110449,
 'get_reveal_mode': 116695,
 'get_reveal_data': 93270,
 'get_full_domain': 66763,
 'get_username': 118054,
 'get_auction_info': 80697,
 'get_telemint_token_name': 69506,
 'get_telemint_auction_state': 122498,
 'get_telemint_auction_config': 129619,
 'get_offer_data': 107279}

# Run transaction & stack step dump

In [43]:
#%load_ext autoreload
#%autoreload 2
from tvm_python import PyTVM, method_name_to_id, code_disasseble
import codecs
from tvm_valuetypes import Cell, deserialize_boc
from dateutil.parser import parse 

transaction = {
        "account_state_state_init_code": "te6ccuEBAQEAYgDEAMD/ACDdIIIBTJe6lzDtRNDXCx/gpPJggwjXGCDTH9Mf0x/4IxO78mPtRNDTH9Mf0//RUTK68qFRRLryogT5AVQQVfkQ8qP4AJMg10qW0wfUAvsA6NEBpMjLH8sfy//J7VRcWvJx",
        "account_state_state_init_data": "te6ccuEBAQEAKgBUAFAAABfFKamjF1EWudQc3BYnnT5cgwKbbuOMnLoj2nddsBRI8MyLn7VbLd5twQ==",
        "gen_utime": "2022-10-29 23:17:14",
        "account_storage_balance_grams": 3503394350113822,
        "in_msg_cell": "te6ccuECAgEAALgAAOYBcAHfiAELXvHRoGvSQCL5tMqMK5vm46jIkBbFPCIaeKYhsLVmxAUv1tIPUGD5rk63+HQIldqZpKMW5papR2c6Je1YCY71+7WGOJg9/+Tmmk+6PG/8ACrE9tk+bE8f6fVUpmDc3uBZTU0YuxrwtygAAL4oHAEAhkIAQGoJHCCLPlTE1kiSIeZM0lzg1DrCKbIT/7fsLkkcrXSovBvI3AAAAAAAAAAAAAAAAAAAAAAAADE5MjY1MjUwNjX5zgaJ",
        "in_msg_body": "te6ccuECAgEAAJUAAKABKgGapfraQeoMHzXJ1v8OgRK7UzSUYtzS1Sjs50S9qwEx3r92sMcTB7/8nNNJ90eN/4AFWJ7bJ82J4/0+qpTMG5vcCympoxdjXhblAAAXxQMBAIZCAEBqCRwgiz5UxNZIkiHmTNJc4NQ6wimyE/+37C5JHK10qLwbyNwAAAAAAAAAAAAAAAAAAAAAAAAxOTI2NTI1MDY1lbIkmg==",
        "in_msg_value_grams": None,
        "in_msg_type": "ext_in_msg_info",
        "in_msg_dest_addr_address_hex": "85AF78E8D035E920117CDA654615CDF371D464480B629E110D3C5310D85AB362",
        "in_msg_dest_addr_workchain_id": 0,
        "lt": 32426921000001,
        "in_msg_created_lt": None
      }


transaction['in_msg_cell'] = {'type': 'cell', 'value': transaction['in_msg_cell']}
transaction['in_msg_body'] = {'type': 'cellSlice', 'value': transaction['in_msg_body']}

tvm = PyTVM(code=transaction['account_state_state_init_code'], 
            data=transaction['account_state_state_init_data'])

tvm.set_c7(unixtime=int(parse(transaction['gen_utime']).timestamp()), 
           translt=str(int(transaction['lt'])), 
           balanceGrams=str(int(transaction['account_storage_balance_grams'])),
           address=str(transaction['in_msg_dest_addr_workchain_id']) + ':' + transaction['in_msg_dest_addr_address_hex'])

tvm.set_stack([
    str(int(transaction['account_storage_balance_grams'])),
    str(int(transaction['in_msg_value_grams'] if transaction['in_msg_value_grams'] else 0)),
    transaction['in_msg_cell'],
    transaction['in_msg_body'],
    -1 if 'ext' in transaction['in_msg_type'] else 0
])

tvm.run_vm()


[3503394350113822,
 0,
 {'type': 'cell',
  'value': 'te6ccuECAgEAALgAAOYBcAHfiAELXvHRoGvSQCL5tMqMK5vm46jIkBbFPCIaeKYhsLVmxAUv1tIPUGD5rk63+HQIldqZpKMW5papR2c6Je1YCY71+7WGOJg9/+Tmmk+6PG/8ACrE9tk+bE8f6fVUpmDc3uBZTU0YuxrwtygAAL4oHAEAhkIAQGoJHCCLPlTE1kiSIeZM0lzg1DrCKbIT/7fsLkkcrXSovBvI3AAAAAAAAAAAAAAAAAAAAAAAADE5MjY1MjUwNjX5zgaJ'}]

In [44]:
list(map(fix_op, tvm.get_ops()))

['SETCP 0',
 'DUP',
 'IFNOTRET',
 'DUP',
 'PUSHINT 85143',
 'EQUAL',
 'PUSHCONT x30ED44D0D70B1F',
 'IFJMP',
 'INC',
 'THROWIF 32',
 'PUSHPOW2 9',
 'LDSLICEX',
 'DUP',
 'LDU 32',
 'LDU 32',
 'LDU 32',
 'NOW',
 'XCHG s1,s3',
 'LEQ',
 'THROWIF 35',
 'PUSH c4',
 'CTOS',
 'LDU 32',
 'LDU 32',
 'LDU 256',
 'ENDS',
 'XCPU s3,s2',
 'EQUAL',
 'THROWIFNOT 33',
 'XCPU s4,s4',
 'EQUAL',
 'THROWIFNOT 34',
 'XCHG s4',
 'HASHSU',
 'XC2PU s0,s5,s5',
 'CHKSIGNU',
 'THROWIFNOT 35',
 'ACCEPT',
 'PUSHCONT x20D74A',
 'PUSHCONT xD307D402FB00',
 'WHILE',
 'DUP',
 'SREFS',
 'implicit RET',
 'LDU 8',
 'LDREF',
 'XCHG s2',
 'SENDRAWMSG',
 'implicit RET',
 'DUP',
 'SREFS',
 'implicit RET',
 'ENDS',
 'SWAP',
 'INC',
 'NEWC',
 'STU 32',
 'STU 32',
 'STU 256',
 'ENDC',
 'POP c4',
 'implicit RET']

In [45]:
tvm.get_stacks()

[[3503394350113822,
  0,
  {'type': 'cell',
   'value': 'te6ccuECAgEAALgAAOYBcAHfiAELXvHRoGvSQCL5tMqMK5vm46jIkBbFPCIaeKYhsLVmxAUv1tIPUGD5rk63+HQIldqZpKMW5papR2c6Je1YCY71+7WGOJg9/+Tmmk+6PG/8ACrE9tk+bE8f6fVUpmDc3uBZTU0YuxrwtygAAL4oHAEAhkIAQGoJHCCLPlTE1kiSIeZM0lzg1DrCKbIT/7fsLkkcrXSovBvI3AAAAAAAAAAAAAAAAAAAAAAAADE5MjY1MjUwNjX5zgaJ'},
  {'type': 'cellSlice',
   'value': 'te6ccuECAgEAAJUAAKABKgGapfraQeoMHzXJ1v8OgRK7UzSUYtzS1Sjs50S9qwEx3r92sMcTB7/8nNNJ90eN/4AFWJ7bJ82J4/0+qpTMG5vcCympoxdjXhblAAAXxQMBAIZCAEBqCRwgiz5UxNZIkiHmTNJc4NQ6wimyE/+37C5JHK10qLwbyNwAAAAAAAAAAAAAAAAAAAAAAAAxOTI2NTI1MDY1lbIkmg=='},
  -1],
 [3503394350113822,
  0,
  {'type': 'cell',
   'value': 'te6ccuECAgEAALgAAOYBcAHfiAELXvHRoGvSQCL5tMqMK5vm46jIkBbFPCIaeKYhsLVmxAUv1tIPUGD5rk63+HQIldqZpKMW5papR2c6Je1YCY71+7WGOJg9/+Tmmk+6PG/8ACrE9tk+bE8f6fVUpmDc3uBZTU0YuxrwtygAAL4oHAEAhkIAQGoJHCCLPlTE1kiSIeZM0lzg1DrCKbIT/7fsLkkcrXSovBvI3AAAAAAAAAAAAAAAAAAAAAAAADE5MjY1MjUwNjX5zgaJ'},
  {'type': 'cellSlice',
   'value': 'te6ccuECAgEAAJUAAKABK